In [3]:
%cd /home/wukailu/latent-nerf

/home/wukailu/latent-nerf


In [4]:
%pwd

'/home/wukailu/latent-nerf'

In [5]:
import os
import torch
from diffusers import DiffusionPipeline

device = "cuda" if torch.cuda.is_available() else "cpu"

pipe = DiffusionPipeline.from_pretrained(
    # "runwayml/stable-diffusion-inpainting",
    # use_auth_token=os.environ.get("API_TOKEN") or True,
    '/home/wukailu/.cache/huggingface/diffusers/cache-runwayml--stable-diffusion-inpainting',
    revision="fp16",
    torch_dtype=torch.float16,
).to(device)

# pipe.save_pretrained('/home/wukailu/.cache/huggingface/diffusers/cache-runwayml--stable-diffusion-inpainting')

Downloading:   0%|          | 0.00/550 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/342 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.75k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/608M [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/810 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.72G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/613 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/167M [00:00<?, ?B/s]

In [18]:
def inpaint(init_image, mask, prompt=""):
    init_image = init_image.convert("RGB").resize((512, 512))
    mask = mask.convert("RGB").resize((512, 512))

    output = pipe(prompt = prompt, image=init_image, mask_image=mask, strength=0.8)
    return output.images[0]

In [19]:
import sys
sys.path.append("/home/wukailu/latent-nerf/src/ControlNet")

In [20]:
import torch
from annotator.util import resize_image, HWC3
from annotator.midas import MidasDetector


apply_midas = MidasDetector()
ROOT_PATH = '/home/wukailu/latent-nerf/src/ControlNet/'

def pred_depth(input_image, detect_resolution):
    with torch.no_grad():
        input_image = HWC3(input_image)
        detected_map, _ = apply_midas(resize_image(input_image, detect_resolution))
        detected_map = HWC3(detected_map)
    return detected_map

In [ ]:
# TODO: warp depth
import os
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

# Util function for loading point clouds|
import numpy as np

# Data structures and functions for rendering
from pytorch3d.structures import Pointclouds
from pytorch3d.vis.plotly_vis import AxisArgs, plot_batch_individually, plot_scene
from pytorch3d.renderer import (
    look_at_view_transform,
    FoVOrthographicCameras,
    PointsRasterizationSettings,
    PointsRenderer,
    PulsarPointsRenderer,
    PointsRasterizer,
    AlphaCompositor,
    NormWeightedCompositor
)

# Setup
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    torch.cuda.set_device(device)
else:
    device = torch.device("cpu")

verts = torch.Tensor(verts).to(device)

rgb = torch.Tensor(rgb).to(device)

R, T = look_at_view_transform(0, 0, 0)
cameras = FoVOrthographicCameras(device=device, R=R, T=T, znear=0.01)
verts = cameras.unproject_points(xy_depth)  # TODO: test this

point_cloud = Pointclouds(points=[verts], features=[rgb])

# Initialize a camera.
R, T = look_at_view_transform(20, 10, 0)
cameras = FoVOrthographicCameras(device=device, R=R, T=T, znear=0.01)

# Define the settings for rasterization and shading. Here we set the output image to be of size
# 512x512. As we are rendering images for visualization purposes only we will set faces_per_pixel=1
# and blur_radius=0.0. Refer to raster_points.py for explanations of these parameters.
raster_settings = PointsRasterizationSettings(
    image_size=512,
    radius = 0.003,
    points_per_pixel = 10
)


# Create a points renderer by compositing points using an alpha compositor (nearer points
# are weighted more heavily). See [1] for an explanation.
rasterizer = PointsRasterizer(cameras=cameras, raster_settings=raster_settings)
renderer = PointsRenderer(
    rasterizer=rasterizer,
    compositor=AlphaCompositor()
)

In [ ]:
# Initialize a camera.
R, T = look_at_view_transform(20, 10, 0)
cameras = FoVOrthographicCameras(device=device, R=R, T=T, znear=0.01)

# Define the settings for rasterization and shading. Here we set the output image to be of size
# 512x512. As we are rendering images for visualization purposes only we will set faces_per_pixel=1
# and blur_radius=0.0. Refer to raster_points.py for explanations of these parameters.
raster_settings = PointsRasterizationSettings(
    image_size=512,
    radius = 0.003,
    points_per_pixel = 10
)


# Create a points renderer by compositing points using an alpha compositor (nearer points
# are weighted more heavily). See [1] for an explanation.
rasterizer = PointsRasterizer(cameras=cameras, raster_settings=raster_settings)
renderer = PointsRenderer(
    rasterizer=rasterizer,
    compositor=AlphaCompositor(background_color=(0, 0, 0))
)

In [ ]:
images = renderer(point_cloud)
plt.figure(figsize=(10, 10))
plt.imshow(images[0, ..., :3].cpu().numpy())
plt.axis("off");